# KI-ENNA: (E)in (N)euronales (N)etz zum (A)usprobieren
Live-Demo basierend auf KI-ENNA 2.0.1 von Prof. Dr. habil. Dennis Klinkhammer

![title](https://github.com/statistical-thinking/ki-enna-demo/blob/main/content/img/image1.png?raw=true)
## Aufgabenstellung
Die Darstellung zeigt drei **verschiedene Pflanzenarten**, deren **Blattlänge und Blattbreite** gemessen worden sind. Während die mit Kreisen dargestellte Planzenart sich eindeutig von den beiden anderen Pflanzenarten unterscheidet, weisen von den anderen Pflanzenarten **einzelne Pflanzen die gleiche Blattlänge und Blattbreite** auf. Daher sind diese Pflanzenarten **schwer zu unterscheiden**. Ein neuronales Netzwerk kann bei der Unterscheidung der ähnlichen Pflanzenarten hilfreich sein. Finde zunächst heraus, **ob ein großes neuronales Netzwerk erforderlich ist**, oder ob ein kleines neuronales Netzwerk die Unterscheidung ebenso gut erledigen kann. In einem weiteren Schritt kannst Du **mit den zugrundliegenden Aktivierungsfunktionen sowie den Epochen und der Lernrate experimentieren**, um die Auswirkungen des Lernverhaltens auf das neuronale Netzwerk beobachten zu können.

## Training des neuronalen Netzwerks
Hier kannst Du die Anzahl an Schichten, Neuronen (und deren Aktivierungsfunktionen) sowie Epochen, aber auch die Lernrate selbst einstellen. Neuronale Netzwerke sind wie kleine Denkmaschinen, **die in Schichten arbeitet**. In die **Eingabeschicht** fließen Zahlen aus Datensätzen ein, die in den **versteckten Schichten** weiter verarbeitet werden. Die **Ausgabeschicht** liefert das Ergebnis. **Neuronen treiben die Denkmaschine an**. Sie rechnen auf der Grundlage der festgelegten **Aktivierungsfunktionen** mit den Zahlen und **schicken Zwischenergebnisse an Neuronen in anderen Schichten weiter**. Aber Achtung: Nicht immer führen mehr Schichten und Neuronen zu einem besseren Ergebnis. Auch falsch ausgewählte Aktivierungsfunktionen können das Ergebnis verfälschen. Damit das funktioniert, braucht ein neuronales Netzwerk Wiederholungen: Eine **Epoche ist ein Durchgang durch alle Übungsaufgaben**. Das macht das neuronale Netzwerk mehrmals, um immer besser zu werden. Die **Lernrate bestimmt, wie schnell das neuronale Netzwerk dabei seine Fehler verbessert**: Ist die Lernrate zu hoch, lernt es zu schnell und macht vielleicht neue Fehler.

In [1]:
# --- Erster Teil des Codes ---
import micropip
await micropip.install('ipywidgets')
import random, math
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# --- Dataset ---
Xtest = [[ 0.81575475, -0.21746808, -0.12904165, -0.65303909],
         [ 0.05761837,  1.59476592,  0.84485761,  1.71304456],
         [ 0.96738203,  0.68864892, -0.00730424, -0.41643072],
         [ 2.02877297,  0.38660992,  2.06223168,  1.00321947],
         [ 1.42226386,  0.99068792,  1.33180724,  0.29339437],
         [ 0.81575475,  0.99068792,  1.21006983,  1.4764362 ],
         [-1.00377258,  0.38660992, -0.49425387, -0.41643072],
         [ 0.05761837, -0.51950708, -0.00730424,  0.29339437],
         [ 0.36087292,  0.38660992,  1.08833242,  1.23982783],
         [ 0.66412748,  0.38660992,  0.35790798,  1.4764362 ],
         [ 0.05761837,  0.08457092,  0.84485761,  0.29339437],
         [-0.70051802, -0.51950708,  0.23617057,  0.53000274],
         [ 0.20924564, -0.21746808,  0.84485761,  1.00321947],
         [-0.24563619,  0.08457092, -0.25077906, -0.65303909],
         [-2.06516352, -1.42562408, -1.95510276, -1.59947255],
         [-1.15539985, -1.42562408, -1.34641572, -1.36286418],
         [ 0.05761837, -1.12358508, -0.00730424, -0.41643072],
         [ 0.20924564,  0.08457092, -0.73772869, -0.88964745],
         [-0.39726347, -0.51950708,  0.23617057, -0.17982236],
         [ 0.5125002 ,  0.08457092, -0.37251647, -0.88964745]]
ytrue = [0,1,0,1,1,1,0,1,1,1,1,1,1,0,0,0,0,0,0,0]

# --- Activation & Loss ---
def sigmoid(x): return 1 / (1 + math.exp(-x))
def sigmoid_derivative(out): return out * (1 - out)
def relu(x): return max(0, x)
def relu_derivative(out): return 1 if out > 0 else 0
def leaky_relu(x): return x if x > 0 else 0.01 * x
def leaky_relu_derivative(out): return 1 if out > 0 else 0.01
def tanh(x): return math.tanh(x)
def tanh_derivative(out): return 1 - out**2
def binary_cross_entropy(pred, y): 
    epsilon = 1e-7
    return - (y * math.log(pred + epsilon) + (1 - y) * math.log(1 - pred + epsilon))
def binary_cross_entropy_derivative(pred, y): 
    epsilon = 1e-7
    return -(y / (pred + epsilon)) + (1 - y) / (1 - pred + epsilon)

# --- Dense Layer ---
def dense_forward(x, w, b, act='relu'):
    pres = [sum(x[i] * w[j][i] for i in range(len(x))) + b[j] for j in range(len(w))]
    outs = []
    for z in pres:
        if act == 'sigmoid':
            outs.append(sigmoid(z))
        elif act == 'relu':
            outs.append(relu(z))
        elif act == 'leaky_relu':
            outs.append(leaky_relu(z))
        elif act == 'tanh':
            outs.append(tanh(z))
    return outs, pres

def dense_backward(x, grad_out, out, pre, w, b, act='relu', lr=0.01):
    grad_in = [0] * len(x)
    for j in range(len(w)):
        if act == 'sigmoid':
            delta = grad_out[j] * sigmoid_derivative(out[j])
        elif act == 'relu':
            delta = grad_out[j] * relu_derivative(pre[j])
        elif act == 'leaky_relu':
            delta = grad_out[j] * leaky_relu_derivative(pre[j])
        elif act == 'tanh':
            delta = grad_out[j] * tanh_derivative(out[j])
        for i in range(len(x)):
            grad_in[i] += w[j][i] * delta
            w[j][i] -= lr * delta * x[i]
        b[j] -= lr * delta
    return grad_in

# --- Visualisierung ---
def html_network_with_connections(layer_sizes, input_dim):
    all_layers = [input_dim] + layer_sizes + [1]
    neuron_size = 20
    margin = 10
    column_spacing = 80
    row_spacing = neuron_size + 20
    radius = neuron_size // 2
    total_width = len(all_layers) * column_spacing
    total_height = max(all_layers) * row_spacing + 2 * margin
    svg = f'<svg width="{total_width}" height="{total_height}" style="position:absolute; top:0; left:0;">'
    positions = []
    for li, n in enumerate(all_layers):
        layer_x = li * column_spacing + column_spacing // 2
        layer = []
        total_layer_height = (n - 1) * row_spacing
        offset_y = (total_height - total_layer_height) // 2
        for ni in range(n):
            layer_y = offset_y + ni * row_spacing
            layer.append((layer_x, layer_y))
        positions.append(layer)
    for i in range(len(positions)-1):
        for x1, y1 in positions[i]:
            for x2, y2 in positions[i+1]:
                svg += f'<line x1="{x1}" y1="{y1}" x2="{x2}" y2="{y2}" stroke="gray" stroke-width="1" />'
    svg += '</svg>'
    html = f'''
    <style>
        .network-wrapper {{
            position: relative;
            width: {total_width}px;
            height: {total_height}px;
        }}
        .network {{
            position: absolute;
            top: 0; left: 0;
            display: flex;
            flex-direction: row;
            justify-content: center;
            height: 100%;
        }}
        .layer {{
            display: flex;
            flex-direction: column;
            justify-content: center;
            align-items: center;
            width: {column_spacing}px;
        }}
        .neuron {{
            width: {neuron_size}px;
            height: {neuron_size}px;
            border: 2px solid black;
            border-radius: 50%;
            background-color: #f2f2f2;
            margin: 10px 0;
            box-sizing: border-box;
        }}
    </style>
    <div class="network-wrapper">
        {svg}
        <div class="network">
    '''
    for n_neurons in all_layers:
        html += '<div class="layer">'
        for _ in range(n_neurons):
            html += '<div class="neuron"></div>'
        html += '</div>'
    html += '</div></div>'
    display(HTML(html))

# --- Speicher ---
trained_model = {}

# --- Training ---
def train_model(X, y, layer_sizes, activations, epochs, lr):
    dims = [len(X[0])] + layer_sizes + [1]
    weights, biases = [], []
    for i in range(len(dims)-1):
        w = [[random.uniform(-0.5, 0.5) for _ in range(dims[i])] for _ in range(dims[i+1])]
        b = [random.uniform(-0.5, 0.5) for _ in range(dims[i+1])]
        weights.append(w)
        biases.append(b)
    loss_trace = []
    for epoch in range(epochs):
        total_loss = 0
        for xi, yi in zip(X, y):
            x = xi
            acts, pres = [], []
            for i in range(len(weights)):
                act = 'sigmoid' if i == len(weights)-1 else activations[i]
                x, pre = dense_forward(x, weights[i], biases[i], act)
                acts.append(x)
                pres.append(pre)
            loss = binary_cross_entropy(acts[-1][0], yi)
            total_loss += loss
            grad = [binary_cross_entropy_derivative(acts[-1][0], yi)]
            for i in reversed(range(len(weights))):
                act = 'sigmoid' if i == len(weights)-1 else activations[i]
                inp = xi if i == 0 else acts[i-1]
                grad = dense_backward(inp, grad, acts[i], pres[i], weights[i], biases[i], act, lr)
        loss_trace.append(total_loss)
    return {
        "weights": weights,
        "biases": biases,
        "loss_trace": loss_trace
    }

# --- Widgets ---
layers_slider = widgets.IntSlider(value=2, min=1, max=5, step=1, description='Schichten:')
neuron_and_activation_controls = widgets.VBox()
epochs_slider = widgets.IntSlider(value=100, min=10, max=500, step=10, description='Epochen:')
lr_slider = widgets.FloatSlider(value=0.05, min=0.001, max=1.0, step=0.01, description='Lernrate:')
train_button = widgets.Button(description="Trainieren")
train_output = widgets.Output()

def update_neuron_sliders(*args):
    count = layers_slider.value
    controls = []
    for i in range(count):
        neuron_slider = widgets.IntSlider(value=4, min=1, max=10, step=1, description=f'{i+1}. Schicht:')
        activation_dropdown = widgets.Dropdown(
            options=['relu', 'leaky_relu', 'sigmoid', 'tanh'],
            value='relu',
            description='mit Funktion:'
        )
        controls.append(widgets.HBox([neuron_slider, activation_dropdown]))
    neuron_and_activation_controls.children = controls

layers_slider.observe(update_neuron_sliders, names='value')
update_neuron_sliders()

def on_train_click(b):
    train_output.clear_output()
    layer_sizes = []
    activations = []
    for control in neuron_and_activation_controls.children:
        neuron_slider, activation_dropdown = control.children
        layer_sizes.append(neuron_slider.value)
        activations.append(activation_dropdown.value)
    epochs = epochs_slider.value
    lr = lr_slider.value
    model = train_model(Xtest, ytrue, layer_sizes, activations, epochs, lr)
    trained_model.clear()
    trained_model.update({
        "weights": model["weights"],
        "biases": model["biases"],
        "loss_trace": model["loss_trace"],
        "layer_sizes": layer_sizes
    })
    with train_output:
        html_network_with_connections(layer_sizes, input_dim=len(Xtest[0]))

train_button.on_click(on_train_click)

display(widgets.VBox([
    widgets.HTML("<b>1. Anzahl an Schichten (N)</b>"),
    layers_slider,
    widgets.HTML("<b>2. Anzahl an Neuronen (N) sowie Aktivierungsfunktionen</b>"),
    neuron_and_activation_controls,
    widgets.HTML("<b>3. Anzahl an Epochen (N)</b>"),
    epochs_slider,
    widgets.HTML("<b>4. Höhe der Lernrate (%)</b>"),
    lr_slider,
    train_button,
    train_output
]))


## Bewertung de neuronalen Netzwerks
Die **Verlust-Funktion ist wie ein Schiedsrichter**. Sie schaut sich die Antworten des neuronalen Netzwerks an und entscheidet, ob das Ergebnis richtig oder falsch ist. Wenn ein neuronales Netzwerk falsch liegt, zeigt die Verlust-Funktion **wie weit die Antwort von der richtigen Lösung entfernt ist**. So lernt das neuronale Netzwerk, was es besser machen muss. Die Antworten eines neuronalen Netzwerks können in einer **Confusion Matrix** zusammengefasst werde. Das ist eine **Tabelle, die zeigt, wie oft ein neuronales Netzwerk etwas richtig oder falsch erkannt hat**. Daraus kann man dann die sogenannte **Accuracy** berechnen, also **wie genau das neuronale Netzwerk insgesamt war**.

In [2]:
# --- Zweiter Teil des Codes ---
from IPython.display import display
eval_button = widgets.Button(description="Bewerten")
eval_output = widgets.Output()

def predict(x, weights, biases):
    for i in range(len(weights)):
        act = 'sigmoid' if i == len(weights)-1 else 'relu'
        x, _ = dense_forward(x, weights[i], biases[i], act)
    return 1 if x[0] > 0.5 else 0

def on_eval_click(b):
    eval_output.clear_output()
    if not trained_model:
        with eval_output:
            print("Bitte zuerst das Modell trainieren.")
        return

    weights = trained_model["weights"]
    biases = trained_model["biases"]
    losses = trained_model["loss_trace"]

    ypred = [predict(xi, weights, biases) for xi in Xtest]
    TP = TN = FP = FN = 0
    for true, pred in zip(ytrue, ypred):
        if true == pred:
            if true == 1: TP += 1
            else: TN += 1
        else:
            if true == 1: FN += 1
            else: FP += 1
    acc = (TP + TN) / len(ytrue)

    with eval_output:
        print("Verlust (jede 10. Epoche):")
        for i in range(9, len(losses), 10):
            print(f"Epoche {i+1:>3}: Verlust = {losses[i]:.4f}")
        print("\nConfusion Matrix:")
        print(f"TN: {TN}  FP: {FP}")
        print(f"FN: {FN}  TP: {TP}")
        print(f"Genauigkeit: {acc:.2f}")

eval_button.on_click(on_eval_click)
display(widgets.VBox([eval_button, eval_output]))

**Prima**, jetzt hast Du ein eigenes neuronales Netzwerk **trainiert und bewertet**!